In [8]:
cities = {
    "New York": {"latitude": 40.7128, "longitude": -74.0060},
    "Los Angeles": {"latitude": 34.0522, "longitude": -118.2437},
    "Chicago": {"latitude": 41.8781, "longitude": -87.6298},
    "Houston": {"latitude": 29.7604, "longitude": -95.3698},
    "Phoenix": {"latitude": 33.4484, "longitude": -112.0740},
    "Philadelphia": {"latitude": 39.9526, "longitude": -75.1652},
    "San Antonio": {"latitude": 29.4241, "longitude": -98.4936},
    "San Diego": {"latitude": 32.7157, "longitude": -117.1611},
    "Dallas": {"latitude": 32.7767, "longitude": -96.7970},
    "Jacksonville": {"latitude": 30.3322, "longitude": -81.6557},
    "Fort Worth": {"latitude": 32.7555, "longitude": -97.3308},
    "Austin": {"latitude": 30.2672, "longitude": -97.7431},
    "San Jose": {"latitude": 37.3382, "longitude": -121.8863},
    "Charlotte": {"latitude": 35.2271, "longitude": -80.8431},
    "Columbus": {"latitude": 39.9612, "longitude": -82.9988},
}


In [10]:
import requests
import pandas as pd

def fetch_nasa_power_data(city, latitude, longitude, start="20180101", end="20221231"):
    url = "https://power.larc.nasa.gov/api/temporal/daily/point"
    params = {
        "start": start,
        "end": end,
        "latitude": latitude,
        "longitude": longitude,
        "community": "RE",
        "parameters": "T2M,RH2M,WS10M,PRECTOTCORR,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN,SG_MID_COZ_ZEN_ANG",
        "format": "JSON"
    }
    
    response = requests.get(url, params=params)
    data = response.json()
    
    if 'properties' in data and 'parameter' in data['properties']:
        parameters = data['properties']['parameter']
        
        # Initialize an empty DataFrame
        df = pd.DataFrame()
        
        # Iterate through each parameter and add it to the DataFrame
        for param, values in parameters.items():
            temp_df = pd.DataFrame.from_dict(values, orient='index', columns=[param])
            df = pd.concat([df, temp_df], axis=1)
        
        # Reset the index and rename the date column
        df.index = pd.to_datetime(df.index)
        df.reset_index(inplace=True)
        df.rename(columns={'index': 'Date'}, inplace=True)
        
        # Save the DataFrame to a CSV file
        df.to_csv(f"{city}_2018_2022.csv", index=False)
        print(f"Data for {city} saved successfully.")
    else:
        print(f"No data found for {city} for the specified parameters and date range.")

# Fetch and save data for each city
for city, coords in cities.items():
    fetch_nasa_power_data(city, coords["latitude"], coords["longitude"])


No data found for New York for the specified parameters and date range.
No data found for Los Angeles for the specified parameters and date range.
No data found for Chicago for the specified parameters and date range.
No data found for Houston for the specified parameters and date range.
No data found for Phoenix for the specified parameters and date range.
No data found for Philadelphia for the specified parameters and date range.
No data found for San Antonio for the specified parameters and date range.
No data found for San Diego for the specified parameters and date range.
No data found for Dallas for the specified parameters and date range.
No data found for Jacksonville for the specified parameters and date range.
No data found for Fort Worth for the specified parameters and date range.
No data found for Austin for the specified parameters and date range.
No data found for San Jose for the specified parameters and date range.
No data found for Charlotte for the specified paramete

In [17]:
import requests
import pandas as pd

# Define the API endpoint and parameters
url = "https://power.larc.nasa.gov/api/temporal/monthly/point"
params = {
    "start": "2018",
    "end": "2021",
    "latitude": "41.54",
    "longitude": "-72.53",
    "community": "re",
    "parameters": "T2M,RH2M,WS10M,PRECTOT,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN",
    "format": "json",
    "header": "true"
}

# Fetch the data
response = requests.get(url, params=params)
data = response.json()

# Extracting the relevant data from the JSON structure
if 'properties' in data and 'parameter' in data['properties']:
    parameters = data['properties']['parameter']
    
    # Creating a DataFrame
    df = pd.DataFrame(parameters)
    
    # Convert the index to a datetime format (YYYYMM)
    df.index = pd.to_datetime(df.index, format='%Y%m')
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'Date'}, inplace=True)
else:
    print("No data found for the specified parameters and date range.")
    df = pd.DataFrame()  # Empty DataFrame as a fallback

# Display the DataFrame
print(df.head())


ValueError: unconverted data remains when parsing with format "%Y%m": "3", at position 12. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [22]:
import requests
import pandas as pd

# Define the API endpoint and parameters
url = "https://power.larc.nasa.gov/api/temporal/monthly/point"
params = {
    "start": "2018",
    "end": "2021",
    "latitude": "41.54",
    "longitude": "-72.53",
    "community": "re",
    "parameters": "T2M,RH2M,WS10M,PRECTOTCORR,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN",
    "format": "json",
    "header": "true"
}

# Fetch the data
response = requests.get(url, params=params)
data = response.json()

# Print the data for inspection
print(data)

# Extracting the relevant data from the JSON structure
if 'properties' in data and 'parameter' in data['properties']:
    parameters = data['properties']['parameter']
    
    # Create an empty DataFrame
    df_list = []
    
    # Iterate through each parameter and add its data to the DataFrame
    for param, values in parameters.items():
        temp_df = pd.DataFrame.from_dict(values, orient='index', columns=[param])
        df_list.append(temp_df)
    
    # Combine all parameter DataFrames
    df = pd.concat(df_list, axis=1)
    
    # Filter out non-date entries and convert to datetime
    valid_dates = df.index.str.match(r'^\d{6}$')
    df = df[valid_dates]
    df.index = pd.to_datetime(df.index, format='%Y%m', errors='coerce')
    df = df.dropna().reset_index()
    df.rename(columns={'index': 'Date'}, inplace=True)
    
    # Save the DataFrame to a CSV file
    df.to_csv('climate_data.csv', index=False)
    print(f"Data saved successfully to climate_data.csv.")
else:
    print("No data found for the specified parameters and date range.")
    df = pd.DataFrame()  # Empty DataFrame as a fallback

# Display the DataFrame
print(df.head())


{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-72.53, 41.54, 87.69]}, 'properties': {'parameter': {'T2M': {'201801': -3.5, '201802': 1.33, '201803': 1.62, '201804': 5.76, '201805': 15.66, '201806': 18.82, '201807': 23.64, '201808': 23.87, '201809': 20.02, '201810': 12.04, '201811': 4.4, '201812': 0.92, '201813': 10.44, '201901': -2.49, '201902': -1.54, '201903': 1.19, '201904': 8.86, '201905': 13.51, '201906': 19.06, '201907': 24.37, '201908': 22.88, '201909': 19.01, '201910': 13.35, '201911': 3.98, '201912': 0.34, '201913': 10.28, '202001': 0.59, '202002': 0.62, '202003': 4.7, '202004': 6.65, '202005': 12.81, '202006': 20.29, '202007': 24.94, '202008': 24.08, '202009': 19.05, '202010': 13.04, '202011': 8.31, '202012': 1.48, '202013': 11.42, '202101': -1.91, '202102': -1.51, '202103': 3.22, '202104': 8.66, '202105': 14.26, '202106': 20.58, '202107': 22.23, '202108': 23.55, '202109': 19.55, '202110': 14.94, '202111': 5.9, '202112': 3.07, '202113': 11.12}, 'RH2M': {'2

In [23]:
import requests
import pandas as pd

# Define the cities and their coordinates
cities = {
    "New York": {"latitude": 40.71, "longitude": -74.01},
    "Los Angeles": {"latitude": 34.05, "longitude": -118.24},
    "Chicago": {"latitude": 41.88, "longitude": -87.63},
    "Houston": {"latitude": 29.76, "longitude": -95.37},
    "Phoenix": {"latitude": 33.45, "longitude": -112.07},
    "Philadelphia": {"latitude": 39.95, "longitude": -75.17},
    "San Antonio": {"latitude": 29.42, "longitude": -98.49},
    "San Diego": {"latitude": 32.72, "longitude": -117.16},
    "Dallas": {"latitude": 32.78, "longitude": -96.80},
    "Jacksonville": {"latitude": 30.33, "longitude": -81.66},
    "Fort Worth": {"latitude": 32.76, "longitude": -97.33},
    "Austin": {"latitude": 30.27, "longitude": -97.74},
    "San Jose": {"latitude": 37.34, "longitude": -121.89},
    "Charlotte": {"latitude": 35.23, "longitude": -80.84},
    "Columbus": {"latitude": 39.96, "longitude": -82.99},
}

# Define the API endpoint and common parameters
url = "https://power.larc.nasa.gov/api/temporal/monthly/point"
common_params = {
    "start": "2018",
    "end": "2021",
    "community": "re",
    "parameters": "T2M,RH2M,WS10M,PRECTOTCORR,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN",
    "format": "json",
    "header": "true"
}

for city, coords in cities.items():
    print(f"Fetching data for {city}...")
    params = common_params.copy()
    params["latitude"] = coords["latitude"]
    params["longitude"] = coords["longitude"]
    
    # Fetch the data
    response = requests.get(url, params=params)
    data = response.json()
    
    # Extracting the relevant data from the JSON structure
    if 'properties' in data and 'parameter' in data['properties']:
        parameters = data['properties']['parameter']
        
        # Create an empty DataFrame
        df_list = []
        
        # Iterate through each parameter and add its data to the DataFrame
        for param, values in parameters.items():
            temp_df = pd.DataFrame.from_dict(values, orient='index', columns=[param])
            df_list.append(temp_df)
        
        # Combine all parameter DataFrames
        df = pd.concat(df_list, axis=1)
        
        # Filter out non-date entries and convert to datetime
        valid_dates = df.index.str.match(r'^\d{6}$')
        df = df[valid_dates]
        df.index = pd.to_datetime(df.index, format='%Y%m', errors='coerce')
        df = df.dropna().reset_index()
        df.rename(columns={'index': 'Date'}, inplace=True)
        
        # Save the DataFrame to a CSV file
        filename = f"{city.replace(' ', '_').lower()}_climate_data.csv"
        df.to_csv(filename, index=False)
        print(f"Data for {city} saved successfully to {filename}.")
    else:
        print(f"No data found for {city} for the specified parameters and date range.")


Fetching data for New York...
Data for New York saved successfully to new_york_climate_data.csv.
Fetching data for Los Angeles...
Data for Los Angeles saved successfully to los_angeles_climate_data.csv.
Fetching data for Chicago...
Data for Chicago saved successfully to chicago_climate_data.csv.
Fetching data for Houston...
Data for Houston saved successfully to houston_climate_data.csv.
Fetching data for Phoenix...
Data for Phoenix saved successfully to phoenix_climate_data.csv.
Fetching data for Philadelphia...
Data for Philadelphia saved successfully to philadelphia_climate_data.csv.
Fetching data for San Antonio...
Data for San Antonio saved successfully to san_antonio_climate_data.csv.
Fetching data for San Diego...
Data for San Diego saved successfully to san_diego_climate_data.csv.
Fetching data for Dallas...
Data for Dallas saved successfully to dallas_climate_data.csv.
Fetching data for Jacksonville...
Data for Jacksonville saved successfully to jacksonville_climate_data.csv.


In [24]:
import requests
import pandas as pd

# Define the cities and their coordinates
cities = {
    "New York": {"latitude": 40.71, "longitude": -74.01},
    "Los Angeles": {"latitude": 34.05, "longitude": -118.24},
    "Chicago": {"latitude": 41.88, "longitude": -87.63},
    "Houston": {"latitude": 29.76, "longitude": -95.37},
    "Phoenix": {"latitude": 33.45, "longitude": -112.07},
    "Philadelphia": {"latitude": 39.95, "longitude": -75.17},
    "San Antonio": {"latitude": 29.42, "longitude": -98.49},
    "San Diego": {"latitude": 32.72, "longitude": -117.16},
    "Dallas": {"latitude": 32.78, "longitude": -96.80},
    "Jacksonville": {"latitude": 30.33, "longitude": -81.66},
    "Fort Worth": {"latitude": 32.76, "longitude": -97.33},
    "Austin": {"latitude": 30.27, "longitude": -97.74},
    "San Jose": {"latitude": 37.34, "longitude": -121.89},
    "Charlotte": {"latitude": 35.23, "longitude": -80.84},
    "Columbus": {"latitude": 39.96, "longitude": -82.99},
}

# Define the API endpoint and common parameters
url = "https://power.larc.nasa.gov/api/temporal/monthly/point"
common_params = {
    "start": "2018",
    "end": "2021",
    "community": "re",
    "parameters": "T2M,RH2M,WS10M,PRECTOTCORR,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN",
    "format": "json",
    "header": "true"
}

# Create an empty DataFrame to store the combined data
combined_df = pd.DataFrame()

for city, coords in cities.items():
    print(f"Fetching data for {city}...")
    params = common_params.copy()
    params["latitude"] = coords["latitude"]
    params["longitude"] = coords["longitude"]
    
    # Fetch the data
    response = requests.get(url, params=params)
    data = response.json()
    
    # Extracting the relevant data from the JSON structure
    if 'properties' in data and 'parameter' in data['properties']:
        parameters = data['properties']['parameter']
        
        # Create an empty DataFrame
        df_list = []
        
        # Iterate through each parameter and add its data to the DataFrame
        for param, values in parameters.items():
            temp_df = pd.DataFrame.from_dict(values, orient='index', columns=[param])
            df_list.append(temp_df)
        
        # Combine all parameter DataFrames
        df = pd.concat(df_list, axis=1)
        
        # Filter out non-date entries and convert to datetime
        valid_dates = df.index.str.match(r'^\d{6}$')
        df = df[valid_dates]
        df.index = pd.to_datetime(df.index, format='%Y%m', errors='coerce')
        df = df.dropna().reset_index()
        df.rename(columns={'index': 'Date'}, inplace=True)
        
        # Add the city name to the DataFrame
        df["City"] = city
        
        # Append the city's data to the combined DataFrame
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    else:
        print(f"No data found for {city} for the specified parameters and date range.")

# Save the combined DataFrame to a single CSV file
combined_df.to_csv("combined_climate_data.csv", index=False)
print("Data for all cities saved successfully to combined_climate_data.csv.")


Fetching data for New York...
Fetching data for Los Angeles...
Fetching data for Chicago...
Fetching data for Houston...
Fetching data for Phoenix...
Fetching data for Philadelphia...
Fetching data for San Antonio...
Fetching data for San Diego...
Fetching data for Dallas...
Fetching data for Jacksonville...
Fetching data for Fort Worth...
Fetching data for Austin...
Fetching data for San Jose...
Fetching data for Charlotte...
Fetching data for Columbus...
Data for all cities saved successfully to combined_climate_data.csv.


In [8]:
import requests
import pandas as pd
import time

# Define the API endpoint for daily data
url = "https://power.larc.nasa.gov/api/temporal/daily/point"

# List of cities with their corresponding latitude and longitude
cities = {
    "New York, New York": {"latitude": 40.71, "longitude": -74.01},
    "Los Angeles, California": {"latitude": 34.05, "longitude": -118.24},
    "Chicago, Illinois": {"latitude": 41.88, "longitude": -87.63},
    "Houston, Texas": {"latitude": 29.76, "longitude": -95.37},
    "Phoenix, Arizona": {"latitude": 33.45, "longitude": -112.07},
    "Philadelphia, Pennsylvania": {"latitude": 39.95, "longitude": -75.17},
    "San Antonio, Texas": {"latitude": 29.42, "longitude": -98.49},
    "San Diego, California": {"latitude": 32.72, "longitude": -117.16},
    "Dallas, Texas": {"latitude": 32.78, "longitude": -96.8},
    "Jacksonville, Florida": {"latitude": 30.33, "longitude": -81.66},
    "Fort Worth, Texas": {"latitude": 32.75, "longitude": -97.33},
    "Austin, Texas": {"latitude": 30.27, "longitude": -97.74},
    "San Jose, California": {"latitude": 37.34, "longitude": -121.89},
    "Charlotte, North Carolina": {"latitude": 35.23, "longitude": -80.84},
    "Columbus, Ohio": {"latitude": 39.96, "longitude": -82.99},
    "Indianapolis, Indiana": {"latitude": 39.77, "longitude": -86.16},
    "San Francisco, California": {"latitude": 37.77, "longitude": -122.42},
    "Seattle, Washington": {"latitude": 47.61, "longitude": -122.33},
    "Denver, Colorado": {"latitude": 39.74, "longitude": -104.99},
    "Oklahoma City, Oklahoma": {"latitude": 35.47, "longitude": -97.52},
    "Nashville, Tennessee": {"latitude": 36.16, "longitude": -86.78},
    "Washington, District of Columbia": {"latitude": 38.91, "longitude": -77.04},
    "El Paso, Texas": {"latitude": 31.76, "longitude": -106.49},
    "Las Vegas, Nevada": {"latitude": 36.17, "longitude": -115.14},
    "Boston, Massachusetts": {"latitude": 42.36, "longitude": -71.06},
    "Detroit, Michigan": {"latitude": 42.33, "longitude": -83.05},
    "Portland, Oregon": {"latitude": 45.52, "longitude": -122.68},
    "Louisville, Kentucky": {"latitude": 38.25, "longitude": -85.76},
    "Memphis, Tennessee": {"latitude": 35.15, "longitude": -90.05},
    "Baltimore, Maryland": {"latitude": 39.29, "longitude": -76.61},
    "Albuquerque, New Mexico": {"latitude": 35.08, "longitude": -106.65},
    "Milwaukee, Wisconsin": {"latitude": 43.04, "longitude": -87.91},
    "Tucson, Arizona": {"latitude": 32.22, "longitude": -110.93},
    "Fresno, California": {"latitude": 36.74, "longitude": -119.79},
    "Sacramento, California": {"latitude": 38.58, "longitude": -121.49},
    "Atlanta, Georgia": {"latitude": 33.75, "longitude": -84.39},
    "Mesa, Arizona": {"latitude": 33.42, "longitude": -111.82},
    "Kansas City, Missouri": {"latitude": 39.1, "longitude": -94.58},
    "Colorado Springs, Colorado": {"latitude": 38.83, "longitude": -104.82},
    "Raleigh, North Carolina": {"latitude": 35.78, "longitude": -78.64},
    "Omaha, Nebraska": {"latitude": 41.26, "longitude": -95.94},
    "Miami, Florida": {"latitude": 25.76, "longitude": -80.19},
    "Virginia Beach, Virginia": {"latitude": 36.85, "longitude": -75.98},
    "Long Beach, California": {"latitude": 33.77, "longitude": -118.19},
    "Oakland, California": {"latitude": 37.8, "longitude": -122.27},
    "Minneapolis, Minnesota": {"latitude": 44.98, "longitude": -93.27},
    "Bakersfield, California": {"latitude": 35.37, "longitude": -119.02},
    "Tulsa, Oklahoma": {"latitude": 36.15, "longitude": -95.99},
    "Tampa, Florida": {"latitude": 27.95, "longitude": -82.46},
    "Arlington, Texas": {"latitude": 32.74, "longitude": -97.11},
    "Aurora, Colorado": {"latitude": 39.73, "longitude": -104.83},
    "Wichita, Kansas": {"latitude": 37.69, "longitude": -97.34},
    "Cleveland, Ohio": {"latitude": 41.5, "longitude": -81.69},
    "New Orleans, Louisiana": {"latitude": 29.95, "longitude": -90.07},
    "Henderson, Nevada": {"latitude": 36.04, "longitude": -114.98},
    "Honolulu, Hawaii": {"latitude": 21.31, "longitude": -157.86},
    "Anaheim, California": {"latitude": 33.83, "longitude": -117.91},
    "Orlando, Florida": {"latitude": 28.54, "longitude": -81.38},
    "Riverside, California": {"latitude": 33.95, "longitude": -117.4},
    "Lexington, Kentucky": {"latitude": 38.04, "longitude": -84.5},
    "Stockton, California": {"latitude": 37.96, "longitude": -121.29},
    "Irvine, California": {"latitude": 33.68, "longitude": -117.82},
    "Corpus Christi, Texas": {"latitude": 27.8, "longitude": -97.4},
    "Cincinnati, Ohio": {"latitude": 39.1, "longitude": -84.51},
    "Santa Ana, California": {"latitude": 33.74, "longitude": -117.87},
    "Greensboro, North Carolina": {"latitude": 36.07, "longitude": -79.79},
    "Pittsburgh, Pennsylvania": {"latitude": 40.44, "longitude": -79.99},
    "Newark, New Jersey": {"latitude": 40.74, "longitude": -74.17},
    "St. Paul, Minnesota": {"latitude": 44.95, "longitude": -93.1},
    "Durham, North Carolina": {"latitude": 35.99, "longitude": -78.9},
    "Lincoln, Nebraska": {"latitude": 40.81, "longitude": -96.68},
    "North Las Vegas, Nevada": {"latitude": 36.2, "longitude": -115.12},
    "Jersey City, New Jersey": {"latitude": 40.73, "longitude": -74.08},
    "Plano, Texas": {"latitude": 33.02, "longitude": -96.71},
    "Anchorage, Alaska": {"latitude": 61.22, "longitude": -149.9},
    "Madison, Wisconsin": {"latitude": 43.07, "longitude": -89.38},
    "Chandler, Arizona": {"latitude": 33.3, "longitude": -111.84},
    "Reno, Nevada": {"latitude": 39.53, "longitude": -119.81},
    "Gilbert, Arizona": {"latitude": 33.35, "longitude": -111.79},
    "St. Louis, Missouri": {"latitude": 38.63, "longitude": -90.2},
    "Chula Vista, California": {"latitude": 32.64, "longitude": -117.08},
    "Buffalo, New York": {"latitude": 42.88, "longitude": -78.88},
    "Fort Wayne, Indiana": {"latitude": 41.08, "longitude": -85.14},
    "Lubbock, Texas": {"latitude": 33.58, "longitude": -101.86},
    "St. Petersburg, Florida": {"latitude": 27.77, "longitude": -82.64},
    "Toledo, Ohio": {"latitude": 41.66, "longitude": -83.56},
    "Laredo, Texas": {"latitude": 27.53, "longitude": -99.49},
    "Port St. Lucie, Florida": {"latitude": 27.27, "longitude": -80.36},
    "Glendale, Arizona": {"latitude": 33.54, "longitude": -112.19},
    "Chesapeake, Virginia": {"latitude": 36.77, "longitude": -76.29},
    "Winston, North Carolina": {"latitude": 36.1, "longitude": -80.26},
    "Irving, Texas": {"latitude": 32.81, "longitude": -96.95},
    "Scottsdale, Arizona": {"latitude": 33.49, "longitude": -111.92},
    "Garland, Texas": {"latitude": 32.91, "longitude": -96.63},
    "Enterprise, Nevada": {"latitude": 36.03, "longitude": -115.21},
    "Arlington, Virginia": {"latitude": 38.88, "longitude": -77.1},
    "Boise, Idaho": {"latitude": 43.62, "longitude": -116.2},
    "Cape Coral, Florida": {"latitude": 26.56, "longitude": -81.95},
    "Frisco, Texas": {"latitude": 33.15, "longitude": -96.82},
    "Richmond, Virginia": {"latitude": 37.54, "longitude": -77.43},
    "Spokane, Washington": {"latitude": 47.66, "longitude": -117.43},
    "Norfolk, Virginia": {"latitude": 36.85, "longitude": -76.29},
    "Huntsville, Alabama": {"latitude": 34.73, "longitude": -86.58},
    "Fremont, California": {"latitude": 37.55, "longitude": -122.03},
    "San Bernardino, California": {"latitude": 34.11, "longitude": -117.29},
    "Tacoma, Washington": {"latitude": 47.25, "longitude": -122.44},
    "Spring Valley, Nevada": {"latitude": 36.11, "longitude": -115.24},
    "Santa Clarita, California": {"latitude": 34.39, "longitude": -118.54},
    "Hialeah, Florida": {"latitude": 25.86, "longitude": -80.28},
    "Modesto, California": {"latitude": 37.64, "longitude": -120.99},
    "McKinney, Texas": {"latitude": 33.2, "longitude": -96.62},
    "Fontana, California": {"latitude": 34.1, "longitude": -117.44},
    "Baton Rouge, Louisiana": {"latitude": 30.45, "longitude": -91.15},
    "Moreno Valley, California": {"latitude": 33.94, "longitude": -117.23},
    "Salt Lake City, Utah": {"latitude": 40.76, "longitude": -111.89},
    "Sioux Falls, South Dakota": {"latitude": 43.55, "longitude": -96.7},
    "Fayetteville, North Carolina": {"latitude": 35.05, "longitude": -78.88},
    "Des Moines, Iowa": {"latitude": 41.59, "longitude": -93.61},
    "Worcester, Massachusetts": {"latitude": 42.26, "longitude": -71.8},
    "Yonkers, New York": {"latitude": 40.93, "longitude": -73.9},
    "Rochester, New York": {"latitude": 43.16, "longitude": -77.61},
    "Little Rock, Arkansas": {"latitude": 34.75, "longitude": -92.29},
    "Grand Prairie, Texas": {"latitude": 32.75, "longitude": -97},
    "Tallahassee, Florida": {"latitude": 30.44, "longitude": -84.28},
    "Amarillo, Texas": {"latitude": 35.22, "longitude": -101.83},
    "Peoria, Arizona": {"latitude": 33.58, "longitude": -112.24},
    "Augusta, Georgia": {"latitude": 33.47, "longitude": -81.97},
    "Knoxville, Tennessee": {"latitude": 35.96, "longitude": -83.92},
    "Columbus, Georgia": {"latitude": 32.46, "longitude": -84.99},
    "Sunrise Manor, Nevada": {"latitude": 36.2, "longitude": -115.07},
    "Vancouver, Washington": {"latitude": 45.64, "longitude": -122.66},
    "Oxnard, California": {"latitude": 34.2, "longitude": -119.18},
    "Overland Park, Kansas": {"latitude": 38.98, "longitude": -94.67},
    "Grand Rapids, Michigan": {"latitude": 42.96, "longitude": -85.67},
    "Birmingham, Alabama": {"latitude": 33.52, "longitude": -86.8},
    "Montgomery, Alabama": {"latitude": 32.37, "longitude": -86.3},
    "Tempe, Arizona": {"latitude": 33.43, "longitude": -111.94},
    "Brownsville, Texas": {"latitude": 25.93, "longitude": -97.48},
    "Providence, Rhode Island": {"latitude": 41.82, "longitude": -71.41},
    "Huntington Beach, California": {"latitude": 33.66, "longitude": -118},
    "Paradise, Nevada": {"latitude": 36.1, "longitude": -115.15},
    "Chattanooga, Tennessee": {"latitude": 35.04, "longitude": -85.31},
    "Akron, Ohio": {"latitude": 41.08, "longitude": -81.52},
    "Clarksville, Tennessee": {"latitude": 36.53, "longitude": -87.35},
    "Fort Lauderdale, Florida": {"latitude": 26.12, "longitude": -80.14},
    "Ontario, California": {"latitude": 34.06, "longitude": -117.65},
    "Glendale, California": {"latitude": 34.14, "longitude": -118.25},
    "Newport News, Virginia": {"latitude": 37.08, "longitude": -76.48},
    "Cary, North Carolina": {"latitude": 35.79, "longitude": -78.78},
    "Mobile, Alabama": {"latitude": 30.69, "longitude": -88.04},
    "Elk Grove, California": {"latitude": 38.41, "longitude": -121.37},
    "Eugene, Oregon": {"latitude": 44.05, "longitude": -123.09},
    "Salem, Oregon": {"latitude": 44.94, "longitude": -123.03},
    "Aurora, Illinois": {"latitude": 41.76, "longitude": -88.32},
    "Santa Rosa, California": {"latitude": 38.44, "longitude": -122.71},
    "Shreveport, Louisiana": {"latitude": 32.53, "longitude": -93.75},
    "Rancho Cucamonga, California": {"latitude": 34.1, "longitude": -117.57},
    "Pembroke Pines, Florida": {"latitude": 26.01, "longitude": -80.34},
    "Springfield, Missouri": {"latitude": 37.21, "longitude": -93.29},
    "Fort Collins, Colorado": {"latitude": 40.56, "longitude": -105.08},
    "Murfreesboro, Tennessee": {"latitude": 35.85, "longitude": -86.39},
    "Oceanside, California": {"latitude": 33.2, "longitude": -117.38},
    "Garden Grove, California": {"latitude": 33.77, "longitude": -117.94},
    "Denton, Texas": {"latitude": 33.21, "longitude": -97.13},
    "Lancaster, California": {"latitude": 34.69, "longitude": -118.15},
    "Surprise, Arizona": {"latitude": 33.63, "longitude": -112.33},
    "Roseville, California": {"latitude": 38.75, "longitude": -121.29},
    "Killeen, Texas": {"latitude": 31.12, "longitude": -97.73},
    "Corona, California": {"latitude": 33.88, "longitude": -117.57},
    "Palmdale, California": {"latitude": 34.58, "longitude": -118.11},
     "Salinas, California": {"latitude": 36.68, "longitude": -121.65},
    "Charleston, South Carolina": {"latitude": 32.78, "longitude": -79.93},
    "Macon, Georgia": {"latitude": 32.84, "longitude": -83.65},
    "Lakewood, Colorado": {"latitude": 39.7, "longitude": -105.08},
    "Paterson, New Jersey": {"latitude": 40.91, "longitude": -74.17},
    "Hollywood, Florida": {"latitude": 26.01, "longitude": -80.15},
    "Alexandria, Virginia": {"latitude": 38.82, "longitude": -77.08},
    "Hayward, California": {"latitude": 37.67, "longitude": -122.08},
    "Springfield, Massachusetts": {"latitude": 42.1, "longitude": -72.59},
    "Kansas City, Kansas": {"latitude": 39.11, "longitude": -94.63},
    "Bellevue, Washington": {"latitude": 47.61, "longitude": -122.2},
    "Sunnyvale, California": {"latitude": 37.37, "longitude": -122.03},
    "Joliet, Illinois": {"latitude": 41.52, "longitude": -88.08},
    "Naperville, Illinois": {"latitude": 41.75, "longitude": -88.16},
    "Olathe, Kansas": {"latitude": 38.88, "longitude": -94.82},
    "Bridgeport, Connecticut": {"latitude": 41.18, "longitude": -73.19},
    "McAllen, Texas": {"latitude": 26.2, "longitude": -98.23},
    "Savannah, Georgia": {"latitude": 32.08, "longitude": -81.1},
    "Escondido, California": {"latitude": 33.12, "longitude": -117.09},
    "Gainesville, Florida": {"latitude": 29.65, "longitude": -82.32},
    "Mesquite, Texas": {"latitude": 32.77, "longitude": -96.6},
    "Waco, Texas": {"latitude": 31.55, "longitude": -97.15},
    "Visalia, California": {"latitude": 36.33, "longitude": -119.29},
    "Thornton, Colorado": {"latitude": 39.87, "longitude": -104.97},
    "Rockford, Illinois": {"latitude": 42.27, "longitude": -89.09},
    "Syracuse, New York": {"latitude": 43.05, "longitude": -76.15},
    "Pasadena, Texas": {"latitude": 29.69, "longitude": -95.2},
    "Columbia, South Carolina": {"latitude": 34, "longitude": -81.03},
    "Pomona, California": {"latitude": 34.05, "longitude": -117.75},
    "Palm Bay, Florida": {"latitude": 28.04, "longitude": -80.61},
    # Add more cities as needed...
}

# Initialize an empty DataFrame to store all data
all_city_data = pd.DataFrame()

# Loop through each city
for city, coords in cities.items():
    # Define parameters for the API request
    params = {
        "start": "20210101",
        "end": "20240731",
        "latitude": round(coords['latitude'], 2),
        "longitude": round(coords['longitude'], 2),
        "community": "RE",
        "parameters": "T2M,T2MWET,RH2M,WS10M,PRECTOTCORR,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN,ALLSKY_SFC_SW_DIFF,CLRSKY_SFC_SW_DIFF,ALLSKY_KT,ALLSKY_SFC_PAR_TOT,CLRSKY_SFC_PAR_TOT,ALLSKY_SFC_UVA,ALLSKY_SFC_UVB,ALLSKY_SFC_UV_INDEX",
        "format": "JSON",
        "header": "true",
        "time-standard": "lst"
    }

    # Fetch the data
    response = requests.get(url, params=params)
    data = response.json()

    # Extract relevant data if available
    if 'properties' in data and 'parameter' in data['properties']:
        parameters = data['properties']['parameter']
        df = pd.DataFrame(parameters)

        # Filter out any invalid entries and convert index to datetime
        valid_dates = df.index.str.match(r'^\d{8}$')
        df = df[valid_dates]
        df.index = pd.to_datetime(df.index, format='%Y%m%d')

        # Reset index and rename columns
        df.reset_index(inplace=True)
        df.rename(columns={'index': 'Date'}, inplace=True)

        # Add city column to identify data from each city
        df['City'] = city

        # Append city data to the combined DataFrame
        all_city_data = pd.concat([all_city_data, df])

        print(f"Data for {city} fetched successfully.")

        # Save the combined data to a CSV file
        all_city_data.to_csv('complete_all_cities_daily_climate_data.csv', index=False)

    else:
        print(f"No data found for {city}.")

    # Wait for 2 minutes before processing the next city
    time.sleep(120)

print("Data collection complete.")

Data for New York, New York fetched successfully.
Data for Los Angeles, California fetched successfully.
Data for Chicago, Illinois fetched successfully.
Data for Houston, Texas fetched successfully.
Data for Phoenix, Arizona fetched successfully.
Data for Philadelphia, Pennsylvania fetched successfully.
Data for San Antonio, Texas fetched successfully.
Data for San Diego, California fetched successfully.
Data for Dallas, Texas fetched successfully.
Data for Jacksonville, Florida fetched successfully.
Data for Fort Worth, Texas fetched successfully.
Data for Austin, Texas fetched successfully.
Data for San Jose, California fetched successfully.
Data for Charlotte, North Carolina fetched successfully.
Data for Columbus, Ohio fetched successfully.
Data for Indianapolis, Indiana fetched successfully.
Data for San Francisco, California fetched successfully.
Data for Seattle, Washington fetched successfully.
Data for Denver, Colorado fetched successfully.
Data for Oklahoma City, Oklahoma fe

Data for Oceanside, California fetched successfully.
Data for Garden Grove, California fetched successfully.
Data for Denton, Texas fetched successfully.
Data for Lancaster, California fetched successfully.
Data for Surprise, Arizona fetched successfully.
Data for Roseville, California fetched successfully.
Data for Killeen, Texas fetched successfully.
Data for Corona, California fetched successfully.
Data for Palmdale, California fetched successfully.
Data for Salinas, California fetched successfully.
Data for Charleston, South Carolina fetched successfully.
Data for Macon, Georgia fetched successfully.
Data for Lakewood, Colorado fetched successfully.
Data for Paterson, New Jersey fetched successfully.
Data for Hollywood, Florida fetched successfully.
Data for Alexandria, Virginia fetched successfully.
Data for Hayward, California fetched successfully.
Data for Springfield, Massachusetts fetched successfully.
Data for Kansas City, Kansas fetched successfully.
Data for Bellevue, Washi